In [1]:
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm #shows a progress meter in a loop

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [2]:
#create empty pandas frame
columns = ['heading','sub_heading', 'time', 'source', 'article', 'tags']
df = pd.DataFrame(columns=columns)
df

,heading,sub_heading,time,source,article,tags


In [3]:
#to get rid of [] related problem and data cleaning while scrapping
sentences = [] #not in use
combined_sentences = '' #in use
import json

#loading file
with open('ten_pages.json') as json_file:
    data = json.load(json_file)

    #going to each article one by one
    for p in data:
        #extracting json data
        heading = p['heading'][0]
        sub_heading = p['sub_heading'][0]
        time = p['time'][0]
        source = p['source'][0]
        article = p['article'][0]
        tags = p['tags']

        #cleaning
        time2 = time.replace("Last Updated : ","").replace(" IST    | Source: ","")
        article = article.replace('"','')

        #each article as a dictionary
        temp_dict_row = {
            "heading" : heading, 
	        "sub_heading" : sub_heading, 
	        "time" : time2, 
	        "source" : source,
            "article" : article,
            "tags" : tags
        }

        #append dictionary as a row in the dataframe
        df = df.append(temp_dict_row , ignore_index=True)

        import re
        article = article.replace(", ",". ")
        combined_sentences = combined_sentences + article +". "
        if len(article.split())>=3 and not(article == ''):
            sentences = sentences + article.split(". ")

In [4]:
df

,heading,sub_heading,time,source,article,tags
0,Economy requires Rs 4.5 lakh crore fiscal support at current juncture: FICCI to FM,"Seeking an ""immediate support"", Reddy said the problem being faced is largely that of liquidity, and immediate release of money stuck in refunds and other government payments to the tune of Rs 2.5...","May 11, 2020 06:38 PM",PTI,"Seeking immediate support for the Indian economy hit by COVID-19, industry body Ficci said an additional fiscal support of Rs 4.5 lakh crore is required at the current juncture besides a quick rel...",#Business#coronavirus#Economy#FICCI#Nirmala Sitharaman
1,ICICI Bank Q4: All eyes on what COVID-19 does to the economy,An uncertain business environment certainly presents risks going ahead not just for banks but all industries.,"May 09, 2020 08:40 PM",Moneycontrol.com,"ICICI Bank’s March-quarter numbers offer us two quick takeaways. One, the COVID-19 uncertainty is developing as the next big risk factor for all banks. In the quarter ended March, ICICI Bank has m...",#Business#Companies#earnings#Economy#ICICI Bank
2,"COVID-19 forces government to rejig borrowing plan, hikes it to Rs 12 lakh crore","The government has also increased the weekly borrowing target to Rs 30,000 crore from Rs 21,000 crore fixed on March 31.","May 08, 2020 08:13 PM",Moneycontrol.com,"To tackle the economic fallout of the COVID-19 pandemic, the government has sharply increased its gross market borrowing programme. The Central government has raised the estimated gross market bor...",#borrowings#coronavirus#coronavirus lockdown#Economy#RBI
3,Bank credit up 6.74% to Rs 102.69 lakh crore; deposits rise 9.82%,"In the fiscal ended March 31, 2020, bank loans had decelerated to 6.14 percent, a near five-decade low, due to slower economic growth, lower demand and as banks remained risk averse.","May 08, 2020 10:34 PM",PTI,"Bank credit and deposits grew by 6.74 percent and 9.82 percent to Rs 102.69 lakh crore and Rs 137.11 lakh crore in the fortnight ended April 24, latest data from the RBI showed. In the year-ago fo...",#Business#Economy#Reserve Bank of India
4,Does India need industries to restart at the cost of diluted labour laws?,Several state governments have decided to set aside provisions of several labour laws for the next two to three years in a bid to restart industrial activity,"May 09, 2020 10:43 AM",Moneycontrol.com,"Shafiq Akhtar, a 34 year-old recently shifted from Mumbai to his native town in Uttar Pradesh to be closer to his family and save more money. While Akhtar was working as an air-conditioner technic...",#Business#Economy#HR
...,...,...,...,...,...,...
388,"After RBI's relief package 2.0, next round of measures could focus on bond market","We continue to expect that the RBI will announce more measures in coming weeks, with the next round largely focusing on the bond market.","Apr 19, 2020 09:42 AM",Moneycontrol.com,"The Reserve Bank of India (RBI)'s relief measures round-two restates the 'whatever it takes' philosophy that policymakers have adopted in the current crisis to minimize the market risks, ease fina...",#Bonds#Economy#Expert Columns#RBI
389,Power Ministry brings new draft of Electricity Amendment Bill,"The draft provides that the ECEA will have sole authority to adjudicate matters related to specific performance of contracts related to purchase or sale of power, between power generation companie...","Apr 18, 2020 06:52 PM",PTI,"The Union power ministry has come out with fourth draft of the Electricity (Amendment) Bill since 2014, which seeks to set up an Electricity Contract Enforcement Authority (ECEA) having power of a...",#Business#Economy#Electricity Amendment Bill#India#policy#Power Ministry
390,Govt's relief packages 'peanuts'; should have been 9% of GDP: Senior Congress leader M Veerappa Moily,Moily's attack on the government came a day after RBI Governor Shaktikanta Das announced a string of relief measures for the stressed banking and financial sector.,"Apr 18, 2020 